In [1]:
import pandas as pd
import sqlite3 
import os

In [2]:
db_file = 'my_app_data.db'

In [3]:
try:
    if os.path.exists(db_file):
        os.remove(db_file)
        print(f"Removed existing database file: '{db_file}'\
                to start fresh.")
except PermissionError as e:
    print(f"PermissionError: Could not remove '{db_file}'.\
          It might be in use by another process. "
          "Please restart your Jupyter kernel "
          "(Kernel -> Restart Kernel) or close any other applications "
          f"accessing this file,then try running this cell again.'Error: {e}")   
    raise 

Removed existing database file: 'my_app_data.db'                to start fresh.


In [4]:
conn = sqlite3.connect(db_file)

In [5]:
cursor = conn.cursor()

In [6]:
create_table_sql = """
CREATE TABLE IF NOT EXISTS users (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    email TEXT UNIQUE NOT NULL
);
"""

In [7]:
cursor.execute(create_table_sql)
print("Table 'users' created or already exists.")

Table 'users' created or already exists.


In [ ]:
# user_data = [
#     ('Alice Smith', 'alice@example.com'),
#     ('Bob Johnson', 'bob@example.com'),
#     ('Charlie Brown', 'charlie@example.com'),
#     ('Alice Smith', 'alice@example.com')  # Duplicate email to show error handling later (though not explicitly handled here)    
# ]

user_data = [
    ('Alice Smith', 'alice@example.com'),
    ('Bob Johnson', 'bob@example.com'),
    ('Charlie Brown', 'charlie@example.com'),      
]


In [9]:
insert_sql = "INSERT INTO users (name, email) VALUES (?, ?);"
try:
    cursor.executemany(insert_sql, user_data)    
    conn.commit()
    print("Sample data inserted successfully.")
except sqlite3.IntegrityError as e:
    print(f"Error inserting data (likely duplicate email): {e}")    
    conn.rollback()
finally:    
    conn.close()
    print("Database connection closed.")

print("\nDatabase 'my_app_data.db' is now ready with sample 'users' data.")
print("You can now proceed to connect and read from it using the next code cells!")


Sample data inserted successfully.
Database connection closed.

Database 'my_app_data.db' is now ready with sample 'users' data.
You can now proceed to connect and read from it using the next code cells!


In [13]:
conn = sqlite3.connect(db_file)

In [11]:
query = "SELECT * FROM users"

In [12]:
df_users_from_db = pd.read_sql(query, conn)

print("Users Data from Database:")
print(df_users_from_db.head())
print("\nUsers Data Info from Database:")
df_users_from_db.info()

Users Data from Database:
   id           name                email
0   1    Alice Smith    alice@example.com
1   2    Bob Johnson      bob@example.com
2   3  Charlie Brown  charlie@example.com

Users Data Info from Database:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      3 non-null      int64 
 1   name    3 non-null      object
 2   email   3 non-null      object
dtypes: int64(1), object(2)
memory usage: 204.0+ bytes


In [14]:
conn.close()
print("\nDatabase connection closed after reading.")


Database connection closed after reading.


########################################################################################################
###################### 2.4.2 Writing Data to a Database: Storing Your Discoveries #######################

In [15]:
new_products_data = {
    'product_id': [101, 102, 103],
    'product_name': ['Smartphone', 'Tablet', 'Smartwatch'],
    'price': [799.99, 499.00, 249.50]
}
df_new_products = pd.DataFrame(new_products_data)

print("New Products DataFrame to be saved:")
print(df_new_products)

New Products DataFrame to be saved:
   product_id product_name   price
0         101   Smartphone  799.99
1         102       Tablet  499.00
2         103   Smartwatch  249.50


In [16]:
conn = sqlite3.connect(db_file)

In [17]:
df_new_products.to_sql('products', conn, if_exists='replace', index=False)
print("\nDataFrame 'df_new_products' saved to 'products' table in the database.")


DataFrame 'df_new_products' saved to 'products' table in the database.


In [19]:
query_products = "SELECT * FROM products"
df_products_from_db = pd.read_sql(query_products, conn)
print("\nProducts Data verified from Database:")
print(df_products_from_db)


Products Data verified from Database:
   product_id product_name   price
0         101   Smartphone  799.99
1         102       Tablet  499.00
2         103   Smartwatch  249.50


In [20]:
conn.close()
print("\nDatabase connection closed after writing and verifying.")



Database connection closed after writing and verifying.
